# Porównanie 3 metod regresji na podstawie 3 różnych zbiorów danych

W niniejszej pracy wykorzystuję metody:
- K najbliższych sąsiądów (KNN)
- Regresji liniowej
- Lasu losowego

Używam następujących zbiorów danych:
- [Melbourne Housing](https://www.kaggle.com/datasets/dansbecker/melbourne-housing-snapshot) 
    - zmienna objaśniana: cena domu
- [Wine Quality](https://www.kaggle.com/datasets/rajyellow46/wine-quality)
    - zmienna objaśniana: jakość wina
-  [Abalon](https://www.kaggle.com/datasets/rodolfomendes/abalone-dataset)
    - zmienna objaśniana: wiek skamieliny

## Załadowanie potrzebnych bibliotek

In [39]:
import random
import os
from joblib import dump, load

import pandas as pd
from ydata_profiling import ProfileReport
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

import sklearn
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import TargetEncoder, StandardScaler, OneHotEncoder
from sklearn.pipeline import make_pipeline

from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.metrics import r2_score


## Stałe

In [2]:
random_state = 0

np.random.seed(random_state)
os.environ["PYTHONHASHSEED"] = str(random_state)
random.seed(random_state)

In [3]:
sklearn.set_config(transform_output="pandas")

## Wczytanie danych

In [4]:
housing = pd.read_csv("../data/melb_data.csv")

### Krótka analiza eksploracyjna danych

In [6]:
# ProfileReport(dataset, title=f"Profiling Report for Housing dataset").to_file(f"../data/housing_EDA.html")

## Preprocessing

In [7]:
housing.sample(5)

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
8505,Williamstown,44 Electra St,4,h,2165000.0,SP,Greg,6/05/2017,8.0,3016.0,...,2.0,2.0,450.0,190.0,1910.0,Hobsons Bay,-37.86100,144.89850,Western Metropolitan,6380.0
5523,Seddon,80 Gamon St,2,h,815000.0,S,Chisholm,30/07/2016,6.6,3011.0,...,1.0,0.0,172.0,81.0,1900.0,Maribyrnong,-37.81000,144.88960,Western Metropolitan,2417.0
12852,Sunshine North,6 Melton Av,3,h,610000.0,SP,Sweeney,16/09/2017,10.5,3020.0,...,1.0,1.0,581.0,NaN,NaN,NaN,-37.76740,144.82421,Western Metropolitan,4217.0
4818,Prahran,16 Park Rd,3,t,1245000.0,PI,Marshall,6/08/2016,4.5,3181.0,...,2.0,1.0,128.0,134.0,2000.0,Stonnington,-37.85260,145.00710,Southern Metropolitan,7717.0
12812,Pascoe Vale,13 Yorkshire St,3,h,1160000.0,S,Nelson,16/09/2017,8.5,3044.0,...,2.0,2.0,480.0,NaN,NaN,NaN,-37.72523,144.94567,Northern Metropolitan,7485.0


In [61]:
housing.isna().sum().sort_values(ascending=False).head(5)

BuildingArea    6450
YearBuilt       5375
CouncilArea     1369
Car               62
Suburb             0
dtype: int64

In [8]:
X = housing.drop(columns="Price")
y = housing["Price"]

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, shuffle=True, random_state=random_state)

In [10]:
imputer = ColumnTransformer([("mode_imputer", SimpleImputer(strategy="most_frequent"), ["YearBuilt", "CouncilArea", "Car"]),
                                ("mean_imputer", SimpleImputer(strategy="mean"), ["BuildingArea"]),
                                ],
                               remainder="passthrough",
                               n_jobs=-1,
                               verbose=True,
                               verbose_feature_names_out=False,
                               )

In [11]:
encoder= ColumnTransformer([("target_encoder", TargetEncoder(categories="auto", target_type="continuous", random_state=random_state), ["SellerG", "Postcode", "Regionname", "CouncilArea", "Postcode", "Suburb"]),
                                ("one_hot_encoder", OneHotEncoder(sparse_output=False), ["Type", "Method"]),
                                     ("drop", "drop", ["Address", "Lattitude", "Longtitude", "Date"])
                                ],
                               remainder="passthrough",
                               n_jobs=-1,
                               verbose=True,
                               verbose_feature_names_out=False,
                               )

In [12]:
preprocessing_pipe = make_pipeline(imputer, encoder, verbose=True)
preprocessing_pipe

Pipeline(steps=[('columntransformer-1',
                 ColumnTransformer(n_jobs=-1, remainder='passthrough',
                                   transformers=[('mode_imputer',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  ['YearBuilt', 'CouncilArea',
                                                   'Car']),
                                                 ('mean_imputer',
                                                  SimpleImputer(),
                                                  ['BuildingArea'])],
                                   verbose=True,
                                   verbose_feature_names_out=False)),
                ('columntransformer-2',
                 ColumnTransformer(n_jobs=-1, remai...
                                   transformers=[('target_encoder',
                                                  TargetEncoder(random_state=0,
                                                                target_type='continuous'),
                                                  ['SellerG', 'Postcode',
                                                   'Regionname', 'CouncilArea',
                                                   'Postcode', 'Suburb']),
                                                 ('one_hot_encoder',
                                                  OneHotEncoder(sparse_output=False),
                                                  ['Type', 'Method']),
                                                 ('drop', 'drop',
                                                  ['Address', 'Lattitude',
                                                   'Longtitude', 'Date'])],
                                   verbose=True,
                                   verbose_feature_names_out=False))],
         verbose=True)

In [13]:
X_train_preprocessed = preprocessing_pipe.fit_transform(X_train, y_train)

[Pipeline]  (step 1 of 2) Processing columntransformer-1, total=   4.9s
[Pipeline]  (step 2 of 2) Processing columntransformer-2, total=   3.1s


# Modelowanie

In [22]:
knn_params = {"n_neighbors": [5, 25, 50],
                "weights": ["uniform", "distance"],
                "leaf_size": [20, 30, 50],
                "p": [1, 2],
                }

random_forest_params = {"n_estimators": [50, 100, 200],
                          # "criterion": ["squared_error", "absolute_error"],
                          "max_depth": [None, 3, 4, 5],
                          "max_features": [None, "sqrt", "log2"],
                          }

Przy wyczerpującym przeszukiwania siatki parametrów w celu znalezienia najlepszej kombinacji parametrów użyjemy walidacji krzyżowej .

[<img src="../img/grid_search_cross_validation.png" alt="drawing" width="400"/>]("../img/grid_search_cross_validation.png")
źródło: https://scikit-learn.org/stable/modules/cross_validation.html

In [31]:
folds = KFold(n_splits=5, shuffle=True, random_state=random_state)

## Housing

### Regresja liniowa

In [23]:
linreg = LinearRegression(n_jobs=-1)

In [36]:
linreg.fit(X_train_preprocessed, y_train)

LinearRegression(n_jobs=-1)

In [49]:
linreg.score(X_train_preprocessed, y_train)

0.6558481241982778

### K najbliższych sąsiadów

In [41]:
search_knn = GridSearchCV(estimator=KNeighborsRegressor(n_jobs=-1),
                           param_grid=knn_params,
                           scoring="r2",
                           n_jobs=-1,
                           refit=True,
                           cv=folds,
                           return_train_score=True,
                           verbose=3,
                           )

In [42]:
%%time
search_knn.fit(X_train_preprocessed, y_train)

Fitting 5 folds for each of 36 candidates, totalling 180 fits
CPU times: total: 3.34 s
Wall time: 2min 26s


GridSearchCV(cv=KFold(n_splits=5, random_state=0, shuffle=True),
             estimator=KNeighborsRegressor(n_jobs=-1), n_jobs=-1,
             param_grid={'leaf_size': [20, 30, 50], 'n_neighbors': [5, 25, 50],
                         'p': [1, 2], 'weights': ['uniform', 'distance']},
             return_train_score=True, scoring='r2', verbose=3)

In [27]:
dump(search_knn, "../models/search_knn_housing.joblib")

['../models/clf_knn_housing.joblib']

In [14]:
search_knn = load("../models/search_knn_housing.joblib")

In [28]:
pd.DataFrame(search_knn.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_leaf_size,param_n_neighbors,param_p,param_weights,params,split0_test_score,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
0,0.085004,0.009719,2.374345,0.016169,20,5,1,uniform,"{'leaf_size': 20, 'n_neighbors': 5, 'p': 1, 'w...",0.289085,...,0.321312,0.021226,13,0.561394,0.555383,0.562088,0.558338,0.552351,0.557911,0.003662
1,0.070268,0.021408,2.240716,0.225363,20,5,1,distance,"{'leaf_size': 20, 'n_neighbors': 5, 'p': 1, 'w...",0.236984,...,0.259464,0.017753,31,0.999966,0.999980,0.999980,0.999982,0.999977,0.999977,0.000006
2,0.058005,0.008415,0.558656,0.040746,20,5,2,uniform,"{'leaf_size': 20, 'n_neighbors': 5, 'p': 2, 'w...",0.315949,...,0.318338,0.011637,16,0.555272,0.553946,0.552493,0.557935,0.548357,0.553601,0.003176
3,0.064276,0.010501,0.421243,0.022991,20,5,2,distance,"{'leaf_size': 20, 'n_neighbors': 5, 'p': 2, 'w...",0.239526,...,0.256448,0.013556,34,0.999965,0.999979,0.999979,0.999981,0.999976,0.999976,0.000006
4,0.053596,0.008265,1.909158,0.021970,20,25,1,uniform,"{'leaf_size': 20, 'n_neighbors': 25, 'p': 1, '...",0.298234,...,0.345487,0.030330,4,0.409644,0.397608,0.396759,0.403235,0.392207,0.399891,0.006006
5,0.061882,0.010882,1.967555,0.186186,20,25,1,distance,"{'leaf_size': 20, 'n_neighbors': 25, 'p': 1, '...",0.266508,...,0.295994,0.016046,28,0.999966,0.999980,0.999980,0.999982,0.999977,0.999977,0.000006
6,0.057075,0.008463,0.487257,0.024399,20,25,2,uniform,"{'leaf_size': 20, 'n_neighbors': 25, 'p': 2, '...",0.304057,...,0.341642,0.025621,7,0.404395,0.392287,0.393044,0.399769,0.386366,0.395172,0.006270
7,0.071238,0.016277,0.613006,0.200227,20,25,2,distance,"{'leaf_size': 20, 'n_neighbors': 25, 'p': 2, '...",0.277561,...,0.300997,0.012957,25,0.999965,0.999979,0.999979,0.999981,0.999976,0.999976,0.000006
8,0.090839,0.033658,2.298931,0.112321,20,50,1,uniform,"{'leaf_size': 20, 'n_neighbors': 50, 'p': 1, '...",0.308106,...,0.346737,0.028913,1,0.383975,0.372636,0.370841,0.381838,0.364872,0.374832,0.007108
9,0.060838,0.010430,2.063907,0.083332,20,50,1,distance,"{'leaf_size': 20, 'n_neighbors': 50, 'p': 1, '...",0.281267,...,0.308720,0.015583,22,0.999966,0.999980,0.999980,0.999982,0.999977,0.999977,0.000006


### Las losowy

In [20]:
search_random_forest = GridSearchCV(estimator=RandomForestRegressor(random_state=random_state),
                                           param_grid=random_forest_params,
                                           scoring="r2",
                                           n_jobs=-1,
                                           refit=True,
                                           cv=folds,
                                           return_train_score=True,
                                         verbose=3,
                                        )

In [21]:
%%time
search_random_forest.fit(X_train_preprocessed, y_train)

Fitting 5 folds for each of 36 candidates, totalling 180 fits
CPU times: total: 49.3 s
Wall time: 4min 26s


GridSearchCV(cv=5, estimator=RandomForestRegressor(random_state=0), n_jobs=-1,
             param_grid={'max_depth': [None, 3, 4, 5],
                         'max_features': [None, 'sqrt', 'log2'],
                         'n_estimators': [50, 100, 200]},
             return_train_score=True, scoring='r2', verbose=3)

In [25]:
# dump(search_random_forest, "../models/search_random_forest.joblib")

['../models/clf_random_forest_housing.joblib']

In [15]:
search_random_forest = load("../models/search_random_forest.joblib")

In [19]:
pd.DataFrame(search_random_forest.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_max_features,param_n_estimators,params,split0_test_score,split1_test_score,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
0,16.458324,1.244014,0.082600,1.730455e-02,None,None,50,"{'max_depth': None, 'max_features': None, 'n_e...",0.752454,0.787597,...,0.787057,0.025964,7,0.971344,0.970206,0.969059,0.968754,0.968599,0.969593,0.001041
1,42.975403,4.175259,0.140961,1.528947e-02,None,None,100,"{'max_depth': None, 'max_features': None, 'n_e...",0.753360,0.791288,...,0.789594,0.026321,2,0.972822,0.971256,0.969773,0.969962,0.969923,0.970747,0.001167
2,99.753294,1.020142,0.290685,3.227211e-02,None,None,200,"{'max_depth': None, 'max_features': None, 'n_e...",0.754286,0.787049,...,0.789707,0.026127,1,0.973392,0.971996,0.970633,0.971019,0.970892,0.971587,0.001014
3,5.700808,0.113172,0.080409,1.577303e-02,None,sqrt,50,"{'max_depth': None, 'max_features': 'sqrt', 'n...",0.749087,0.797681,...,0.786796,0.025488,8,0.970633,0.969018,0.968340,0.968775,0.967570,0.968867,0.001011
4,11.245025,0.121422,0.143661,8.211669e-03,None,sqrt,100,"{'max_depth': None, 'max_features': 'sqrt', 'n...",0.751377,0.799468,...,0.788256,0.024313,3,0.972216,0.970264,0.969218,0.969791,0.969107,0.970119,0.001128
5,24.050381,0.532603,0.264900,9.987629e-03,None,sqrt,200,"{'max_depth': None, 'max_features': 'sqrt', 'n...",0.750490,0.798738,...,0.788155,0.024813,5,0.972699,0.971374,0.970069,0.970719,0.970362,0.971045,0.000935
6,6.049705,0.143496,0.085884,5.523919e-03,None,log2,50,"{'max_depth': None, 'max_features': 'log2', 'n...",0.749087,0.797681,...,0.786796,0.025488,8,0.970633,0.969018,0.968340,0.968775,0.967570,0.968867,0.001011
7,11.571395,0.157325,0.134375,7.654013e-03,None,log2,100,"{'max_depth': None, 'max_features': 'log2', 'n...",0.751377,0.799468,...,0.788256,0.024313,3,0.972216,0.970264,0.969218,0.969791,0.969107,0.970119,0.001128
8,22.422577,0.182216,0.281250,1.711650e-02,None,log2,200,"{'max_depth': None, 'max_features': 'log2', 'n...",0.750490,0.798738,...,0.788155,0.024813,5,0.972699,0.971374,0.970069,0.970719,0.970362,0.971045,0.000935
9,2.120406,0.144851,0.031251,4.264961e-07,3,None,50,"{'max_depth': 3, 'max_features': None, 'n_esti...",0.542148,0.583038,...,0.574204,0.025723,36,0.588992,0.570127,0.581009,0.591562,0.581516,0.582641,0.007489


## Analiza

Na tym tym posiadam już dostrojone, finalne modele. Teraz zostaną one porównane wg następujących metryk:

- MAE
- MAPE
- R2

In [4]:
# TODO: wybrać finalne metryki